In [2]:
import os
import sys
from bs4 import BeautifulSoup
import requests
import lxml
import numpy as np
import csv
import json
import numpy
import copy
import pandas as pd
import re
# Import package dependencies

ModuleNotFoundError: No module named 'lxml'

In [246]:
# Configure the path to the directory containing the data files
domain = 'https://www.itexams.com'
exam_code = '/exam/AZ-900?'
url = domain + exam_code

In [247]:
def parse_url_from_text(text):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex, text)      
    return [x[0] for x in url]

In [248]:
def remove_url_from_text(text):
    text = re.sub(r'http\S+', '', text)
    text = text.replace('Reference:', '')
    return text

In [249]:
def blank_character_cleaner(text):
    text = text.replace('\n', '').replace('\t', '').lstrip()
    return text

In [250]:
def fetch_data(url):
    response = requests.get(url)
    return response.text

In [251]:
def parse_data(data):
    soup = BeautifulSoup(data, 'lxml')
    each_question_card = soup.findChildren('div', {'id': 'accordion'}, recursive=True)[0]
    each_question_card_element = each_question_card.findChildren('div', {'class': 'card'}, recursive=True)
    return each_question_card_element

In [252]:
data = parse_data(fetch_data(url))
print("Number of questions : {}".format(len(data)))


Number of questions : 15


In [253]:
data_output = list()
for i in range(len(data)):

    data_payload = data[i].findChildren(recursive=True)

    question_id = blank_character_cleaner(data_payload[0].findChildren(recursive=True)[0].text.replace(' ', ''))
    question_payload = blank_character_cleaner(data_payload[4].findChildren(recursive=True)[0].find_all('p')[2].text)
    question_statement = blank_character_cleaner(data_payload[4].findChildren(recursive=True)[0].find_all('p')[0].text)
    question_answer_key = blank_character_cleaner(data_payload[4].findChildren(recursive=True)[0].find_all('p')[1].text)
    # question_answer_image = data_payload[4].findChildren(recursive=True)[0].find_all('img')[1]['src']
    question_answer_key = question_answer_key.replace('Answer :', '').lstrip()
    image_payload = data_payload[4].findChildren(recursive=True)[0].find_all('img')

    question_image = ''
    answer_image = ''
    if len(image_payload) != 0:
        if (len(image_payload) == 1):
            question_image = domain + image_payload[0]['src']
        if (len(image_payload) == 2):
            question_image = domain + image_payload[0]['src']
            answer_image = domain + image_payload[1]['src']

    reference_links = parse_url_from_text(question_payload)

    question_explaination = remove_url_from_text(question_payload)

    reference_link_1 = ''
    reference_link_2 = ''
    if (len(reference_links) != 0):
        if (len(reference_links) == 1):
            reference_link_1 = reference_links[0]
        if (len(reference_links) == 2):
            reference_link_1 = reference_links[0]
            reference_link_2 = reference_links[1]
    
    data_parsed = {
        "question_id": question_id,
        "question_statement": question_statement,
        "question_answer_key": question_answer_key,
        "question_image": question_image,
        "answer_image": answer_image,
        "question_explaination": question_explaination,
        "reference_link_1": reference_link_1,
        "reference_link_2": reference_link_2
    }

    
    data_output.append(data_parsed)
    # print("QuestionId : {}\n".format(question_id))
    # print('Question_Statement : {}\n'.format(question_statement))
    # print('Question_Image : {}\n'.format(question_image))
    # print('Question_Answer_key : {}\n'.format(question_answer_key))
    # print('Question_Answer_image : {}\n'.format(answer_image))
    # print('Question_Explaination : {}\n'.format(question_explaination))
    # print('Reference Links : \n', reference_links)


In [254]:
# print(data_output)
# data_list = json.dumps(data_output)
# print(type(data_output))
df = pd.json_normalize(data_output)
df.head(len(data_output))
df.to_csv('data.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'data.csv'